In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [1]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

scaler = preprocessing.StandardScaler()
scaler.fit_transform(df)
df = pd.DataFrame(scaler.transform(df))

df['Class'] = targets
x_train = df.iloc[:, 0:16]
y_train = df.iloc[:, 16]
learning_rates = [0.3, 0.03, 0.003]
activation_functions = ['logistic', 'relu', 'tanh']
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


best_combination = []
best_acc = -1
best_pred = []
best_test = -1
test = -1
with open('best_mlp.txt', 'w') as f:
  print('Starting Tests...')
  for first in range(10, 16):
    for second in range(3, 16):
      for lr in learning_rates:
        for af in activation_functions:
          test = test + 1
          print(f'Itializing test {test}...')
          print(f'Test Parameters: First Layer Neurons={first}, Second Layers Neurons={second}, Learning Rate={lr}, Activation Functions={af}.')
          classifier = MLPClassifier(activation=af, solver='adam', alpha=1e-5, hidden_layer_sizes=(first, second), random_state=1, verbose=False, learning_rate_init=lr, tol=1e-3, max_iter=500)
          y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
          print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
          print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
          print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
          print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')
          acc = np.mean(y_pred['test_acc'])*100
          prec = np.mean(y_pred['test_prec'])*100
          rec = np.mean(y_pred['test_recall'])*100
          f1 = np.mean(y_pred['test_f1'])*100
          f.write(f'{test},{first},{second},{lr},{acc},{prec},{rec},{f1}')
          print
          if acc > best_acc:
            print(f'Test {test} performed better than test {best_test}. Adjusting new best results...')
            best_combination = [first, second, lr, af]
            best_acc = acc
            best_pred = y_pred
            best_test = test
            best_pred = [acc, prec, rec, f1]

print('Best Result')
print(f'Test Parameters: First Layer Neurons={best_combination[0]}, Second Layers Neurons={best_combination[1]}, Learning Rate={best_combination[2]}, Activation Functions={best_combination[3]}.')
print('Results...')
print('Acurácia Média: ' + '%.2f' % (best_pred[0]) + '%')
print('Precisão Média: ' + '%.2f' % (best_pred[1]) + '%')
print('Revocação Média: ' + '%.2f' % (best_pred[2]) + '%')
print('F1-Score Médio: ' + '%.2f' % (best_pred[3]) + '%')

NameError: name 'fetch_ucirepo' is not defined